In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import pi
from itertools import islice
import cv2
import tensorflow as tf
import pickle

In [2]:
#reading the data set from data.txt

dataset_path='H:\Case Study\Self Driving Case Study\Autopilot-TensorFlow-master\Autopilot-TensorFlow-master\driving_dataset'
training_path=os.path.join(dataset_path,'data.txt')

split=0.8
X=[]
y=[]

#islice(text, start, stop, LIMIT) --> ('ABCDEF', 2, None) --> 'C D E F'  -->https://docs.python.org/2/library/itertools.html
with open(training_path) as f:
    for line in islice(f,None):
        path,angle=line.strip().split()
        full_path=os.path.join(dataset_path,path)
        X.append(full_path)
        y.append((float(angle)*pi)/180)
        
y=np.array(y)

print("Finished Preprocessing")

#split it with 80/20

split_index=int(len(y)*0.8)
y_train=y[:split_index]
y_val=y[split_index:]

Finished Preprocessing


### 3.3 Reading from pickle file

In [3]:
with open('X_train', 'rb') as f:
    X_train = pickle.load(f)

In [4]:
with open('X_val', 'rb') as f:
    X_val = pickle.load(f)

In [5]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(36324, 66, 200, 3)
(9082, 66, 200, 3)
(36324,)
(9082,)


## 4. Preparing data for modelling

In [6]:
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]
channels = X_train.shape[3]

print(img_rows, img_cols, channels)

66 200 3


In [7]:
import tensorflow.keras.backend as K

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], channels, img_rows, img_cols).astype('float32')
    X_val = X_val.reshape(X_val.shape[0], channels, img_rows, img_cols).astype('float32')
    img_size = (channels, img_rows, img_cols)
    
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, channels).astype('float32')
    X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, channels).astype('float32')
    img_size = (img_rows, img_cols, channels)

In [8]:
X_train = X_train/255.0
X_val = X_val/255.0

In [9]:
print(X_train.shape)
print(X_val.shape)
print(img_size)

(36324, 66, 200, 3)
(9082, 66, 200, 3)
(66, 200, 3)


## 5. Building model from Nvidia Architecture with Hyperparameter Optimization(Using regularization and Batch Normalization)

In [10]:
import tensorflow.keras.backend as K

def custom_activation(x):
    return tf.multiply(tf.math.atan(x), 2) #  arctan range is [-pi/2, pi/2]. so we have to get the angle between [-pi, pi] so we are multiplying by 2

In [19]:
#to refer the model architecture: https://developer.nvidia.com/blog/deep-learning-self-driving-cars/
#to use atan as a activation function refer: https://www.tensorflow.org/api_docs/python/tf/math/atan
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
img_input = tf.keras.Input(img_size)
x = tf.keras.layers.Conv2D(filters=24, kernel_size=(5,5), strides=(2,2), padding='valid',activation='relu',name='conv-1')(img_input)
x = tf.keras.layers.Conv2D(filters=36, kernel_size=(5,5), strides=(2,2), padding='valid',activation='relu', name='conv-2')(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters=48, kernel_size=(5,5), strides=(2,2), padding='valid',activation='relu',kernel_regularizer =tf.keras.regularizers.l1( l=0.01), name='conv-3')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='valid',activation='relu',kernel_regularizer =tf.keras.regularizers.l1( l=0.01), name='conv-4')(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='valid',activation='relu',kernel_regularizer =tf.keras.regularizers.l1( l=0.01), name='conv-5')(x)
x = tf.keras.layers.Flatten(name='flatten')(x)
x = tf.keras.layers.Dense(1164, activation='relu', name='dense_layer-1')(x)
x = tf.keras.layers.Dropout(0.2, name='dropout-1')(x)
x = tf.keras.layers.Dense(100, activation='relu', name='dense_layer-2')(x)
x = tf.keras.layers.Dropout(0.2, name='dropout-2')(x)
x = tf.keras.layers.Dense(50, activation='relu', name='dense_layer-3')(x)
x = tf.keras.layers.Dropout(0.2, name='dropout-3')(x)
x = tf.keras.layers.Dense(10, activation='relu', name='dense_layer-4')(x)
x = tf.keras.layers.Dropout(0.2, name='dropout-4')(x)
output = tf.keras.layers.Dense(1, activation=custom_activation, name='output')(x) #note we have used atan to use tan inverse activation

model_2 = tf.keras.models.Model(inputs=img_input, outputs=output, name='model-1')

## 6. Compiling and fit the model

In [20]:
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')

In [21]:
batch_size = 100
Epochs = 30

In [22]:
from datetime import datetime
from tensorflow.keras.callbacks import  ModelCheckpoint

checkpoint_file_name = 'H:\Case Study\Self Driving Case Study\Checkpoints\envideahyper\envidiahyper-weights.{epoch:02d}-{val_loss:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(checkpoint_file_name, monitor='val_loss', verbose=1, save_best_only=True)

callbacks = [model_checkpoint]

In [23]:
history_2 = model_2.fit(X_train, y_train, batch_size=batch_size, epochs=Epochs, validation_data=(X_val, y_val), callbacks=callbacks, verbose=2)

Train on 36324 samples, validate on 9082 samples
Epoch 1/30

Epoch 00001: val_loss improved from inf to 11.56089, saving model to H:\Case Study\Self Driving Case Study\Checkpoints\envideahyper\envidiahyper-weights.01-11.56.hdf5
36324/36324 - 262s - loss: 23.0090 - val_loss: 11.5609
Epoch 2/30

Epoch 00002: val_loss improved from 11.56089 to 1.38812, saving model to H:\Case Study\Self Driving Case Study\Checkpoints\envideahyper\envidiahyper-weights.02-1.39.hdf5
36324/36324 - 247s - loss: 5.3965 - val_loss: 1.3881
Epoch 3/30

Epoch 00003: val_loss improved from 1.38812 to 0.63725, saving model to H:\Case Study\Self Driving Case Study\Checkpoints\envideahyper\envidiahyper-weights.03-0.64.hdf5
36324/36324 - 253s - loss: 0.6456 - val_loss: 0.6373
Epoch 4/30

Epoch 00004: val_loss improved from 0.63725 to 0.26436, saving model to H:\Case Study\Self Driving Case Study\Checkpoints\envideahyper\envidiahyper-weights.04-0.26.hdf5
36324/36324 - 254s - loss: 0.3290 - val_loss: 0.2644
Epoch 5/30

Ep